In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from datasets import load_dataset
from tigerscore import TIGERScorer
# set up scorer
scorer = TIGERScorer(model_name="TIGER-Lab/TIGERScore-7B-V1.2", quantized=False)
# load the dataset
dataset = load_dataset("TIGER-Lab/MetricInstruct")
num_few_examples = 10
tasks = dataset["train_mix"]['task'][0:num_few_examples]
insts = dataset["train_mix"]['instruction'][0:num_few_examples]
input_contexts = dataset["train_mix"]['input_context'][0:num_few_examples]
hypo_output = dataset["train_mix"]['hypo_output'][0:num_few_examples]
# scoring
results = scorer.score(tasks, insts, input_contexts, hypo_output, batch_size=8)
scores = [result["score"] for result in results] # List of float scores
print(scores)
print(results[0]) # associated explanation texts


/home/dongfu/miniconda3/envs/tigerscore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-26 18:00:16,615] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


TIGERScore Batch Scoring: 100%|██████████| 2/2 [00:43<00:00, 21.84s/it]

[-0.0, -0.0, -6.0, -0.0, -5.0, -6.0, -1.0, -1.0, -1.0, -1.0]
{'num_errors': 1, 'score': -0.0, 'errors': {'error_0': {'location': 'participate in the DFB Presidency', 'aspect': 'Accuracy', 'explanation': "The verb 'participate' is not the most accurate translation of 'mitarbeiten' in this context. A more accurate translation would be 'work with'.", 'severity': 'Minor', 'score_reduction': '0.5'}}, 'raw_output': 'You are evaluating errors in a model-generated output for a given instruction.\nInstruction: \nTranslate the following text from German to English.\nDer künftige EM-Cheforganisator Philipp Lahm soll laut Grindel im DFB-Präsidium mitarbeiten.\n\nModel-generated Output: \nAccording to Grindel, the future head of the European Championships, Philipp Lahm, is to participate in the DFB Presidency.\n\nFor each error you give in the response, please also elaborate the following information:\n- error location (the words that are wrong in the output)\n- error aspect it belongs to.\n- expla